# AVITO PROJECT

## Team Member

* Sandikha Rahardi - РИМ 130908

## Load Data

### Import Package

In [16]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
from sklearn.model_selection import train_test_split

### Set Dataframe

In [17]:
dataset_filename = "assets/avito_dataset.xlsx"
main_df = pd.read_excel(dataset_filename, sheet_name="Sheet1")

## Data Cleaning

### Important Column

In [18]:
# Define important column to predict the price
important_columns = [
    'Цена', 'Метро', 'Метро2', 'Метро. Время', 
    'Район', 'Сегодня просмотров', 'Всего просмотров', 
    'Продавец. Компания', 'Статус', 'Количество комнат',
    'Общая площадь, число', 'Этаж', 'Жилая площадь, число',
    'Санузел', 'Балкон или лоджия', 'Окна', 'Ремонт', 
    'Способ продажи', 'Мебель', 'Вид сделки', 'Техника',
    'Тип дома', 'Этажей в доме', 'Год постройки',
    'Пассажирский лифт', 'Парковка', 'Двор', 'Грузовой лифт', 'В доме',
    'Тёплый пол', 'Дополнительно', 'Отделка',
    'Площадь кухни, число', 'Тип комнат'
]
print("Total original column : " , main_df.columns.__len__())
print("Total important column : " , important_columns.__len__())
main_df = main_df[important_columns]

Total original column :  71
Total important column :  34


In [19]:
print(main_df.shape)
main_df.isnull().sum() / main_df.shape[0] * 100

(19803, 34)


Цена                     0.000000
Метро                    0.489825
Метро2                  47.856385
Метро. Время            47.856385
Район                   52.310256
Сегодня просмотров      33.979700
Всего просмотров         1.449275
Продавец. Компания       0.000000
Статус                   0.000000
Количество комнат        0.000000
Общая площадь, число     0.000000
Этаж                     0.000000
Жилая площадь, число    35.853154
Санузел                 40.301974
Балкон или лоджия       32.692016
Окна                    56.824724
Ремонт                  60.793819
Способ продажи          44.866939
Мебель                  81.755290
Вид сделки              60.157552
Техника                 90.930667
Тип дома                13.750442
Этажей в доме            0.000000
Год постройки           69.848003
Пассажирский лифт       60.177751
Парковка                55.769328
Двор                    59.041559
Грузовой лифт           66.696965
В доме                  89.577337
Тёплый пол    

### Clean the dataset

#### Remove duplicated dataset

In [20]:
print("Duplicated data : ", main_df.duplicated().sum())
main_df = main_df.drop_duplicates().reset_index(drop=True)
print("Duplicated data (now) : ", main_df.duplicated().sum())

Duplicated data :  38
Duplicated data (now) :  0


#### Clean about 'Метро'

In [21]:
main_df.dropna(subset=['Метро'], inplace=True)

def preprocess_metro_time(row):
    if(type(row) == str):
        row = row.replace("мин.","").strip()
        if(row.__contains__("от")):
            row = float(row.replace("от","").strip())
        elif(row.__contains__("до")):
            row = float(row.replace("до","").strip())
        elif(row.__contains__("–")):
            row = (int(row.split("–")[0]) + int(row.split("–")[1])) / 2
    return row

main_df['Метро. Время'] = main_df['Метро. Время'].apply(preprocess_metro_time)
mean_metro_time = round(pd.to_numeric(main_df['Метро. Время']).mean(), 1)
main_df['Метро. Время'].fillna(mean_metro_time, inplace=True)
main_df['Метро2'] = main_df['Метро2'].apply(lambda x: "Нет" if pd.isna(x) else "Есть")

#### Clean about 'Район'

In [22]:
main_df['Район'].fillna("Другой", inplace=True)

#### Clean about 'Просмотров'

In [23]:
main_df['Сегодня просмотров'].fillna(0, inplace=True)
main_df['Всего просмотров'].fillna(0, inplace=True)

#### Clean about 'Количество комнат'

In [24]:
main_df['Количество комнат'] = main_df['Количество комнат'].apply(lambda x: 0 if x == 'Студия' else x)

max_room = pd.to_numeric(main_df['Количество комнат'], errors='coerce').max()
mean_room = pd.to_numeric(main_df['Количество комнат'], errors='coerce').mean()

main_df['Количество комнат'] = main_df['Количество комнат'].apply(lambda x: max_room if x == 'многокомнатные' else x)
main_df['Количество комнат'] = main_df['Количество комнат'].apply(lambda x: round(mean_room) if x == 'Своб. планировка' else x)

main_df['Количество комнат'].value_counts()

Количество комнат
2.0    6552
1.0    6328
3.0    3610
0.0    2601
4.0     493
5.0      67
6.0       9
7.0       8
Name: count, dtype: int64

> Note: Студия does not have room ( 0 room )

#### Clean about 'Общая площадь, число'

In [25]:
main_df['Общая площадь, число'] = main_df['Общая площадь, число'].apply(lambda x: float(x.replace(",", ".")) if type(x) == str else int(x))

#### Clean about 'Этаж'

In [26]:
# Clean dataset 'Этаж' of 'Этажей в доме'
main_df['Этаж'] = main_df['Этаж'].apply(lambda x: int(x.split("из")[0].strip()))

#### Clean about 'Жилая площадь, число'

In [27]:
main_df['Жилая площадь, число'].fillna(0, inplace=True)

#### Clean about 'Санузел'

In [28]:
main_df['Санузел'].fillna('Другой', inplace=True)

#### Clean about 'Балкон или лоджия'

In [29]:
main_df['Балкон или лоджия'].fillna('Другой', inplace=True)

#### Clean about 'Окна'

In [30]:
main_df['Окна'].fillna('Другой', inplace=True)

#### Clean about 'Ремонт'

In [31]:
main_df['Ремонт'].fillna('Другой', inplace=True)

#### Clean about 'Способ продажи'

In [32]:
main_df['Способ продажи'].fillna('Другой', inplace=True)

#### Clean about 'Мебель'

In [33]:
main_df['Мебель'].fillna('Другой', inplace=True)

#### Clean about 'Вид сделки'

In [34]:
main_df['Вид сделки'].fillna('Другой', inplace=True)

#### Clean about 'Техника'

In [35]:
main_df['Техника'].fillna(0, inplace=True)

def preprocess_tech(row):
    if(type(row) == str):
        if(row.__contains__(",")):
            row = len(row.split(","))
        else: 
            row = 1
    return row
main_df['Техника'] = main_df['Техника'].apply(preprocess_tech)

#### Clean about 'Тип дома'

In [36]:
main_df['Тип дома'].fillna('Другой', inplace=True)

#### Clean about 'Год постройки'

In [37]:
# Change null value with median
print("Data Null :" , main_df['Год постройки'].isna().sum())
print("Data Mode :" , main_df['Год постройки'].mode()[0])
main_df['Год постройки'].fillna(main_df['Год постройки'].median(), inplace=True)

Data Null : 13744
Data Mode : 2023.0


#### Clean about 'Пассажирский лифт'

In [38]:
# Change null value with mode
main_df['Пассажирский лифт'] = main_df['Пассажирский лифт'].apply(lambda x: 0 if x == 'Нет' else x)
print("Data Null :" , main_df['Пассажирский лифт'].isna().sum())
print("Data Mode :" , main_df['Пассажирский лифт'].mode()[0])
main_df['Пассажирский лифт'].fillna(main_df['Пассажирский лифт'].mode()[0], inplace=True)

Data Null : 11835
Data Mode : 1.0


#### Clean about 'Парковка'

In [39]:
main_df['Парковка'].fillna(0, inplace=True)

def preprocess_parking(row):
    if(type(row) == str):
        if(row.__contains__(",")):
            row = len(row.split(","))
        else: 
            row = 1
    return row
main_df['Парковка'] = main_df['Парковка'].apply(preprocess_parking)

#### Clean about 'Двор'

In [40]:
main_df['Двор'].fillna(0, inplace=True)

def preprocess_yard(row):
    if(type(row) == str):
        if(row.__contains__(",")):
            row = len(row.split(","))
        else: 
            row = 1
    return row
main_df['Двор'] = main_df['Двор'].apply(preprocess_yard)

#### Clean about 'Грузовой лифт'

In [41]:
# Change null value with mode
main_df['Грузовой лифт'] = main_df['Грузовой лифт'].apply(lambda x: 0 if x == 'Нет' else x)
print("Data Null :" , main_df['Грузовой лифт'].isna().sum())
print("Data Mode :" , main_df['Грузовой лифт'].mode()[0])
main_df['Грузовой лифт'].fillna(main_df['Грузовой лифт'].mode()[0], inplace=True)

Data Null : 13123
Data Mode : 1.0


#### Clean about 'В доме'

In [42]:
main_df['В доме'].fillna('Другой', inplace=True)

#### Clean about 'Тёплый пол'

In [43]:
main_df['Тёплый пол'].fillna('Нет', inplace=True)

#### Clean about 'Дополнительно'

In [44]:
main_df['Дополнительно'].fillna('Другой', inplace=True)

#### Clean about 'Отделка'

In [45]:
main_df['Отделка'].fillna('Другой', inplace=True)

#### Clean about 'Площадь кухни, число'

In [46]:
main_df['Площадь кухни, число'] = main_df['Площадь кухни, число'].apply(lambda x: float(x.replace(",", ".")) if type(x) == str else x)
print("Data Null :" , main_df['Площадь кухни, число'].isna().sum())
print("Data Mode :" , main_df['Площадь кухни, число'].mode()[0])
main_df['Площадь кухни, число'].describe()

Data Null : 3777
Data Mode : 6.0


count    15891.000000
mean        13.997533
std          7.044985
min          2.000000
25%          8.200000
50%         13.900000
75%         18.000000
max        100.000000
Name: Площадь кухни, число, dtype: float64

In [47]:
rounded_number = round(main_df['Площадь кухни, число'].mode()[0], 1)
main_df['Площадь кухни, число'].fillna(rounded_number, inplace=True)

#### Clean about 'Тип комнат'

In [48]:
main_df['Тип комнат'].fillna('Другой', inplace=True)

### Split Dataset

In [49]:
# Split the data into training (80%), testing (15%), and validation (5%)
train_df, temp_df = train_test_split(main_df, test_size=0.2, random_state=42)
test_df, val_df = train_test_split(temp_df, test_size=0.25, random_state=42)
print(train_df.shape)
print(test_df.shape)
print(val_df.shape)

(15734, 34)
(2950, 34)
(984, 34)


### Divided into Categorical and Numerical dataset

In [50]:
numerical_columns = ['Цена', 'Метро. Время', 'Сегодня просмотров', 'Всего просмотров', 'Количество комнат', 'Общая площадь, число', 'Этаж', 'Жилая площадь, число', 'Этажей в доме', 'Год постройки', 'Пассажирский лифт', 'Парковка', 'Двор', 'Грузовой лифт', 'Площадь кухни, число', 'Техника']
categorical_columns = ['Метро', 'Метро2', 'Район', 'Продавец. Компания', 'Статус', 'Санузел', 'Балкон или лоджия', 'Окна', 'Ремонт', 'Способ продажи', 'Мебель', 'Вид сделки', 'Тип дома', 'В доме', 'Тёплый пол', 'Дополнительно', 'Отделка', 'Тип комнат']

train_num_df = train_df[numerical_columns]
train_cat_df = train_df[categorical_columns]
test_num_df = test_df[numerical_columns]
test_cat_df = test_df[categorical_columns]
val_num_df = val_df[numerical_columns]
val_cat_df = val_df[categorical_columns]